## Intial analysis of water data collected in Alicante

In [1]:
import os
import sys
import math
import pandas as pandas
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from statistics import mean

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from modules import water_monitoring

In [2]:
instance = water_monitoring.WaterMonitoringInstance()
instance.read_data("../data/Benalua Consumption/Benalua 2013.csv", ";")
instance.transform_data("alicante", True)
instance.make_time_features("CA009-52")

data = instance.get_data_frame()
data

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning:

Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.



,CA009-52,Data-1(Sensor1),Data-2(Sensor1),Minute,Hour,Day,Month,Year
0,2013-01-02 03:25:36,25,98,25,3,2,1,2013
1,2013-03-13 10:32:12,78,76678,32,10,13,3,2013
2,2013-03-13 10:32:41,78,8452,32,10,13,3,2013
3,2013-03-13 10:33:12,77,53304,33,10,13,3,2013
4,2013-03-13 10:33:41,74,85148,33,10,13,3,2013
...,...,...,...,...,...,...,...,...
793751,2013-12-31 15:11:25,75,4491,11,15,31,12,2013
793752,2013-12-31 15:11:55,74,881,11,15,31,12,2013
793753,2013-12-31 15:12:27,76,3143,12,15,31,12,2013
793754,2013-12-31 15:12:57,81,9696,12,15,31,12,2013


In [8]:
coef_arr = instance.calculate_correlation("Data-1(Sensor1)")
coef_arr

Attribute CA009-52 can't be correlated. invalid type promotion

Column with the most correlation is: Hour. Pearson coef: 0.3790


C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2534: RuntimeWarning:

invalid value encountered in true_divide

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2535: RuntimeWarning:

invalid value encountered in true_divide

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning:

invalid value encountered in greater

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning:

invalid value encountered in less

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1912: RuntimeWarning:

invalid value encountered in less_equal



[('Data-2(Sensor1)', -0.07361906982354366),
 ('Day', -0.001738344347952742),
 ('Minute', -0.00029852286813139957),
 ('Month', 0.04982170922352415),
 ('Hour', 0.37898870898162035),
 ('Year', nan)]

In [4]:
instance.get_nan_rows()

,CA009-52,Unnamed: 1,CT005-52,Unnamed: 3,Unnamed: 4
793756,NaN,NaN,NaN,NaN,NaN


In [5]:
"""fig = px.line(data, x="CA009-52", y="Data-1(Sensor1)", height=600)
fig.show()""";

In [6]:
# instance.plot_two_columns( "CA009-52", "Data-1(Sensor1)")

In [7]:
# instance.interactive_two_columns_plot("CA009-52", "Data-1(Sensor1)")